# Face Recognition Module 2

---

## Recap

In [1]:
# Ignore FutureWarning warnings that may pop up from importing following libraries
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# Libraries from last part of the project
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from glob import glob
from collections import defaultdict

# Importing functions from the last part of the project's ipynb file as a library for this part
import import_ipynb
from Facial_Recognition_Module import *

importing Jupyter notebook from Facial_Recognition_Module.ipynb


In [3]:
''' Display all images from a given list '''

def show_all_images(img_list, columns=4):
    
    # Shape matplotlib subplot
    size = len(img_list)
    rows = int(size / columns)
    if size % columns  != 0 or rows == 0:
        rows += 1
    
    figsize = (10, 5)
    fig, axs = plt.subplots(rows, columns, figsize=figsize)
    
    # Convert grayscale images to BGR
    if img_list[0].shape[-1] != 3:
        for idx, img in enumerate(img_list):
            img_list[idx] = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    
    # Draw all the images if more than one row can be drawn with given column number
    i = 0
    if rows > 1:
        for row in range(rows):
            for column in range(columns):
                if i >= len(img_list):
                    axs[row][column].axis('off')
                    i += 1
                    continue
                img = img_list[i]
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                _ = axs[row][column].imshow(img)
                _ = axs[row][column].axis('off')
                i += 1
    
    # Draw all images if images only take up one row
    else:
        for column in range(columns):
            if i >= len(img_list):
                    _ = axs[column].axis('off')
                    i += 1
                    continue
            img = img_list[i]
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            _ = axs[column].imshow(img)
            _ = axs[column].axis('off')
            i += 1 
        
        
    _ = plt.axis('off')
    plt.show()

---

## Rescaling Images to Image Size Median for Uniformity

In [4]:
''' Resize the face images to the median face image size from the list'''

def normalize_face_image_size(gray_face_list):
    
    # Gather all the width/height dimensions from the face images
    x_size_list = y_size_list = []
    for face in gray_face_list:
        x_size_list.append(face.shape[0])
        y_size_list.append(face.shape[1])
    
    # Find the median from the width/height dimensions
    x_median = int(np.median(x_size_list))
    y_median = int(np.median(y_size_list))
    dim = (x_median, y_median)
    
    # Resize the face image to the median face image size
    rescale_face_list = []
    for face in gray_face_list:
        if face.shape[0] == 0 or face.shape[1] == 0:
            gray_face_list.remove(face)
            continue
        rescale_face = cv2.resize(face, dim)
        rescale_face_list.append(rescale_face)
        
    return rescale_face_list

In [5]:
''' Draw side by side image comparisons using matplotlib '''

def compare_two_images(img_1, img_2):
    
    # Create subplots plot for a 1x2 image subplot
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    
    # Display the first image
    _ = axs[0].imshow(img_1, cmap='gray')
    _ = axs[0].axis('off')
    
    # Display the second image
    _ = axs[1].imshow(img_2, cmap='gray')
    _ = plt.axis('off')
    plt.show()

In [6]:
''' Apply preprocessing steps to provided face image list '''

def preprocess_images(gray_face_list, preprocess_steps):
    
    # Make a copy of the face image list
    process_face_list = gray_face_list.copy()
    
    # If there are no preprocessing steps provided, return the original list
    if len(preprocess_steps) == 0:
        return process_face_list
    
    # Apply the preprocessing steps provided in the list to the list
    for step in preprocess_steps: 
        process_face_list = step(process_face_list)
    return process_face_list

In [7]:
''' Detect faces from original images and apply preprocessing steps
    Organize the images into defaultdicts where value entries are lists containing original and face images
'''

def prepare_image_dictionaries(original_list, label_list, preprocess_steps):
    
    img_dict = defaultdict(list)
    face_dict = defaultdict(list)
    gray_face_list = []
    
    # Organize the original image and face lists in the img_dict/face_dict by their corresponding labels
    for img, label in zip(original_list, label_list):
        faces, _ = detect_faces(img)
        if faces is not None:
            img_dict[label].append(img)
            face_dict[label].append(faces[0])
    
    # Add all the faces from the face dictionary to a gray_face_list super list
    for _, face_list in face_dict.items():
        for face in face_list:
            gray_face_list.append(face)
    
    # Process all the faces in the super list
    process_face_list = preprocess_images(gray_face_list, preprocess_steps)
    
    # Store the processed faces to the face dictionary
    start = end = 0
    for label, img_list in img_dict.items():
        start, end = end, end + len(img_list)
        face_dict[label] = process_face_list[start:end]
  
    return img_dict, face_dict

---

## Gaussian Blur

In [8]:
'''Apply a Gaussian blur filter to the face list'''

def apply_blur(gray_face_list):
    blur_face_list = []
    for face in gray_face_list:
        if face.shape[0] == 0 or face.shape[1] == 0:
            gray_face_list.remove(face)
            continue
        blur = cv2.GaussianBlur(face, (7, 7), 0)
        blur_face_list.append(blur)
    return blur_face_list

---

## Augmenting Image Data

In [9]:
import Augmentor
import shutil
import random

In [10]:
''' Create a clean training directory for the specified character '''

def make_training_directory(character, path):
    training_path = path + '/' + character + '/training'

    # Create a fresh training directory for each character
    try:
        _ = os.mkdir(training_path)
    except FileExistsError:
        _ = shutil.rmtree(training_path)
        _ = os.mkdir(training_path)
    
    return training_path

In [11]:
''' Apply Augmentor to images stored in project directory to generate num_samples more images'''

def augment_images(path_name, num_samples=100, random_state=None):
    random.seed(random_state)
    path_to_images = glob(path_name) 
    for path in path_to_images:

        # Delete output directories if they exist already
        output_path = path + '/output'
        try:
            shutil.rmtree(output_path)
        except FileNotFoundError:
            pass

        # Implement Augmentor pipeline to each character image directory w. random prob of each step occurring
        p = Augmentor.Pipeline(path)
        p.rotate(random.uniform(0.25, 0.75), max_left_rotation=10, max_right_rotation=10)
        p.shear(random.uniform(0.25, 0.75), max_shear_left=1, max_shear_right=0.01)
        p.shear(random.uniform(0.25, 0.75), max_shear_left=0.01, max_shear_right=1)
        p.flip_left_right(random.uniform(0.25, 0.75))
        
        # Create specified number of random samples
        _ = p.sample(num_samples)

In [12]:
'''Perform a train-test split on the given ith-fold with image augmentation on training images'''

def train_augment_test_split(character_dict, project_path, data_dict, face_dict, i,
                             num_samples, folds, random_state=None):
    
    test_img_list = []
    test_label_list = np.array([])
    
    # Train-test split on each individual character
    for (label, img_list), (_, face_list) in zip(data_dict.items(), face_dict.items()):
        list_len = len(img_list)
        test_sample_len = int(1/folds * list_len)

        # Test Split on face images
        test_split = face_list[i*test_sample_len : (i+1)*test_sample_len]
        test_label_list = np.append(test_label_list, np.full(len(test_split), label))
        test_img_list = test_img_list + test_split

        # Train Split on original images
        train_split = img_list[:i*test_sample_len] + img_list[(i+1)*test_sample_len:]

        # Save training images to training directory
        character = character_dict[label]
        training_path = make_training_directory(character, project_path)
        for img_num, img in enumerate(train_split):
            img_path = '{}/{}.png'.format(training_path, img_num)
            _ = cv2.imwrite(img_path, img)

        # Augment the face images from the training directory and save augmentation to output directory
        _ = augment_images(training_path, num_samples, random_state)
    
    return test_img_list, test_label_list

---

## FaceNet

In [13]:
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
from sklearn.svm import SVC
from sklearn.preprocessing import Normalizer

Using TensorFlow backend.


In [14]:
'''
The model provided for face embedding will be the FaceNet model generated by Hiroi Taniai.
The output is a 128-D embedding vector of the face provided
'''
def embed_face(face, model, required_size=(160,160)):
    
    # Turn face into RGB acceptable format for FaceNet model
    if face.shape[-1] != 3:
        img = cv2.cvtColor(face, cv2.COLOR_GRAY2RGB)
    else:
        img = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
    
    # Resize the face image to the required 160x160 FaceNet image size    
    img = cv2.resize(img, required_size)
    img = img.astype('float32')
    
    # Standardize the pixel values
    mean, std = img.mean(), img.std()
    img = (img - mean) / std
    
    # Turn the processed face into a sample that can be used for embedding
    sample = np.expand_dims(img, axis=0)
    
    # Embed the face to a 128 element vector using the FaceNet model
    embedding = model.predict(sample)
    
    # Return the embedded vector
    return embedding[0]

In [15]:
''' Edit the generate_confusion_matrix function to work with image embeddings from FaceNet 
    (changes provided in else-statement)
'''

def generate_confusion_matrix(character_dict, test_img_list, test_label_list, 
                              face_recognizer, test_X=None, test_y=None):
    
    # Generate initial confusion matrix DataFrame
    character_list = []
    for _, character in character_dict.items():
        character_list.append(character)
    
    confusion_matrix = pd.DataFrame(index=character_list, columns=character_list)
    confusion_matrix.loc[:, :] = 0
    
    pred_img_df = pd.DataFrame(index=character_list, columns=character_list).astype('object')
    for i in range(pred_img_df.shape[0]):
        for j in range(pred_img_df.shape[1]):
            pred_img_df.iloc[i, j] = []
    
    # Make predictions on test images
    if (test_X is None) and (test_y is None):
        for test_img, true_label in zip(test_img_list, test_label_list):

            # What to do if the test images are in color 
            if (test_img.shape[-1] == 3):
                # Make prediction on test image and add to pred_face_list
                pred_label_list, pred_rect_list = predict(test_img, face_recognizer)

                if pred_rect_list is not None:
                    pred_img = draw_character_labels(test_img, character_dict, 
                                                     pred_label_list, pred_rect_list)

                # Populate confusion matrix based on prediction
                actual_character = character_dict[true_label]
                pred_character = character_dict[pred_label_list[0]]

                confusion_matrix.loc[pred_character, actual_character] += 1
                pred_img_df.loc[pred_character, actual_character].append(pred_img)

            # What to do if the test image is in grayscale
            else:
                # Predict on the grayscale face provided
                pred_label = face_recognizer.predict(test_img)

                # Populate confusion matrix based on prediction
                actual_character = character_dict[true_label]
                pred_character = character_dict[pred_label[0]]

                confusion_matrix.loc[pred_character, actual_character] += 1
                pred_img_df.loc[pred_character, actual_character].append(test_img)
    
    # Make prediction on image vectors
    else:
        
        # Predict on the testing image set
        pred_y = face_recognizer.predict(test_X)
        
        for ((idx, pred_label), true_label) in zip(enumerate(pred_y), test_y):
            
            # Add point to confusion matrix
            pred_character = character_dict[pred_label]
            actual_character = character_dict[true_label]
            confusion_matrix.loc[pred_character, actual_character] += 1
            
            # Add test image to DataFrame
            img = test_img_list[idx].copy()
            pred_img_df.loc[pred_character, actual_character].append(img)
    
    return confusion_matrix, pred_img_df

In [16]:
''' Edit the CV function to work on image embeddings (face_recognizer takes on the form of FaceNet model)'''

def cross_validate(original_list, label_list, face_recognizer, character_dict, 
                    project_path, preprocess_steps=[normalize_face_image_size],
                       num_samples=100, folds=5, random_state=None):
    
    pred_img_df_list = []
    confusion_matrix_list = []
    f1_average_list = []
    t_list = []
    
    # Shuffle the images
    cv_img_list, cv_label_list = shuffle_original(original_list, label_list, random_state)
    
    # Prepare an image dictionary with images where the LBPH could detect a face
    data_dict, original_face_dict = prepare_image_dictionaries(cv_img_list, 
                                                               cv_label_list, 
                                                               [])
        
    for i in range(folds):
            
        # Augment the training data and create testing image data
        test_face_list, test_label_list = train_augment_test_split(character_dict,
                                                                  project_path,
                                                                  data_dict, 
                                                                  original_face_dict, i,
                                                                  num_samples, 
                                                                  folds, random_state)
            
        # Retrieve augmented image data for training the classifier
        training_img_path = project_path + '/*/training/output/*.png' 
        _, train_img_list, train_label_list = prepare_data(training_img_path, -4)
        
        # Gather faces from augmented training data
        _, augment_face_dict = prepare_image_dictionaries(train_img_list, 
                                                       train_label_list, 
                                                       preprocess_steps)
        
        # Convert the training faces to 128-element embedded FaceNet vectors
        embed_augment_face_dict = defaultdict(list)
        for key, face_list in augment_face_dict.items():
            embedding_face_list = []
            for face in face_list:
                if face is None or face.shape[0] == 0 or face.shape[1] == 0:
                    continue
                embedding = embed_face(face, face_recognizer)
                embedding_face_list.append(embedding)
            embed_augment_face_dict[key] = embedding_face_list
        
        
        # Convert the testing faces to 128-element embedded FaceNet vectors
        test_face_dict = defaultdict(list)
        for img, label in zip(test_face_list, test_label_list):
            test_face_dict[label].append(img)
        embed_test_face_dict = defaultdict(list)
        test_face_list = []
        test_face_label_list = np.array([])
        for label, face_list in test_face_dict.items():
            embedding_face_list = []
            for face in face_list:
                if face is None or face.shape[0] == 0 or face.shape[1] == 0:
                    continue
                embedding = embed_face(face, face_recognizer)
                embedding_face_list.append(embedding)
                test_face_list.append(face)
                test_face_label_list = np.append(test_face_label_list, label)
            embed_test_face_dict[label] = embedding_face_list

        
        # Generate the final training embedding list and labels from embed_augment_face_dict
        train_X = []
        train_y = np.array([])
        for (label, embed_list) in embed_augment_face_dict.items():
            for embed in embed_list:
                train_X.append(embed)
            train_y = np.append(train_y, np.full(len(embed_list), label))
        train_X = np.asarray(train_X)
        
        # Generate the final testing embedding list and labels from the embed_test_face_dict
        test_X = []
        test_y = np.array([])
        for (label, embed_list) in embed_test_face_dict.items():
            for embed in embed_list:
                test_X.append(embed)
            test_y = np.append(test_y, np.full(len(embed_list), label))
        test_X = np.asarray(test_X)
        
        
        # Cast training label list as integer list and train the classifier
        train_y = train_y.astype(int)
        test_y = test_y.astype(int)
        
        # Normalize input vectors
        norm = Normalizer(norm='l2')
        train_X = norm.transform(train_X)
        test_X = norm.transform(test_X)
        
        # Train a linear SVM for facial recognition
        model = SVC(kernel='linear', probability=True)
        model.fit(train_X, train_y)
        
        # Generate the confusion matrix by predicting test images
        confusion_matrix, pred_img_df = generate_confusion_matrix(character_dict, 
                                                                  test_face_list, test_face_label_list, 
                                                                  model, test_X, test_y)
        confusion_matrix_list.append(confusion_matrix)
        pred_img_df_list.append(pred_img_df)
        
        # Gather f1 score information from data gathered in testing phase
        f1_score_list = generate_f1_scores(confusion_matrix, character_dict)
        character_average_f1 = np.mean(f1_score_list)
        f1_average_list.append(character_average_f1)
        t_list.append(test_face_list)
        
    # Average the f1 scores together
    cv_average_f1 = np.mean(f1_average_list)
    
    return pred_img_df_list, confusion_matrix_list, f1_average_list, cv_average_f1, t_list

---

## Implementing Multi-Task Convolutional Neural Network (MTCNN) Facial Detection

In [17]:
from mtcnn.mtcnn import MTCNN

In [18]:
'''Detect faces from a photograph using MTCNN'''

def detect_faces(img, detector):
    
    rect_coord_list = []
    faces_list = []

    # Run facial detection on the image
    faces = detector.detect_faces(img)
    if len(faces)==0:
        return None, None
    
    # Create face and rectangle coordinate lists from facial detection
    for face in faces:
        (x, y, w, h) = face['box']
        if h == 0 or w == 0:
            return None, None
        faces_list.append(img[y:y+h, x:x+w])
        rect_coord_list.append((x, y, w, h))
    
    # Return face image and coordinates where the face was found
    return faces_list, rect_coord_list

In [19]:
''' Edit the prepare_image_dictionaries function to use MTCNN when detecting faces '''

def prepare_image_dictionaries(original_list, label_list, preprocess_steps):
    
    img_dict = defaultdict(list)
    face_dict = defaultdict(list)
    super_face_list = []
    detector = MTCNN()
    
    # Organize the original image and face lists by their corresponding labels
    for img, label in zip(original_list, label_list):
        faces, _ = detect_faces(img, detector)
        if faces is not None:
            img_dict[label].append(img)
            face_dict[label].append(faces[0])
    
    # Add all the faces from the face dictionary to a gray_face_list super list
    for _, face_list in face_dict.items():
        for face in face_list:
            super_face_list.append(face)
    
    # Process all the faces in the super list
    process_face_list = preprocess_images(super_face_list, preprocess_steps)
    
    # Store the processed faces to the face dictionary
    start = end = 0
    for label, img_list in img_dict.items():
        start, end = end, end + len(img_list)
        face_dict[label] = process_face_list[start:end]
  
    return img_dict, face_dict

---

## Final Functions

In [20]:
'''Augment the images from the training path that you want to train with'''

def augment_training_images(character_dict, project_path, img_dict,
                             num_samples=100, random_state=None):
        
    # Gather all the original images for a character each individual character
    for (label, img_list) in img_dict.items():

        # Train on all the original images
        train_split = img_list

        # Save training images to training directory
        character = character_dict[label]
        training_path = make_training_directory(character, project_path)
        for img_num, face in enumerate(train_split):
            img_path = '{}/{}.png'.format(training_path, img_num)
            _ = cv2.imwrite(img_path, face)

        # Augment the face images from the training directory and save augmentation to output directory
        _ = augment_images(training_path, num_samples, random_state)

In [21]:
'''Extract features from images using FaceNet'''
def embed_training_images(project_path, feature_extractor):
     
        # Retrieve augmented image data for img embedding
        training_img_path = project_path + '/*/training/output/*.png' 
        _, train_img_list, train_label_list = prepare_data(training_img_path, -4)
        
        # Gather faces from augmented training data
        _, augment_face_dict = prepare_image_dictionaries(train_img_list, 
                                                       train_label_list, 
                                                       [])
        
        # Convert the training faces to 128-element embedded FaceNet vectors
        embed_augment_face_dict = defaultdict(list)
        for key, face_list in augment_face_dict.items():
            embedding_face_list = []
            for face in face_list:
                if face is None or face.shape[0] == 0 or face.shape[1] == 0:
                    continue
                embedding = embed_face(face, feature_extractor)
                embedding_face_list.append(embedding)
            embed_augment_face_dict[key] = embedding_face_list
        
        return embed_augment_face_dict

In [22]:
''' 
testing_img_class used to categorize images with their corresponding faces, face coordinates, and embedded vectors
'''

class testing_img_class(object):
    
    def __init__(self, img, face_list, rect_coord_list):
        self.img = img
        self.face_list = face_list
        self.rect_coord_list = rect_coord_list
        self.embedded_face_list = self.prediction_list = []
    
    def add_embedding_list(self, embedding_list):
        self.embedded_face_list = embedding_list
    
    def add_predictions(self, prediction_list):
        self.prediction_list = prediction_list

In [23]:
''' Generates list of test_img_class class types that store img and faces / coordinates of where face was found '''

def detect_faces_from_testing_imgs(test_img_list):
    
    test_img_class_list = []
    detector = MTCNN()
    
    # Organize the original image and face lists by their corresponding labels
    for img in test_img_list:
        face_list, rect_coord_list = detect_faces(img, detector)
        if face_list is not None and rect_coord_list is not None:
            temp = testing_img_class(img, face_list, rect_coord_list)
            test_img_class_list.append(temp)
  
    return test_img_class_list

In [24]:
''' Adds all the embedded vectors to the test img class list'''
def embed_testing_images(test_img_class_list, feature_extractor):
        
    # Convert the testing faces to 128-element embedded FaceNet vectors
    for item in test_img_class_list:
        embedding_face_list = []
        if item.face_list is None:
            continue
        for face in item.face_list:
            embedding = embed_face(face, feature_extractor)
            if embedding is None:
                continue
            embedding_face_list.append(embedding)
        item.add_embedding_list(embedding_face_list)

In [25]:
'''Train SVM on embeddings from augmented image data'''
def train_SVM(embed_augment_face_dict):
    
    # Generate the final training embedding list and labels from embed_augment_face_dict
    train_X = []
    train_y = np.array([])
    for (label, embed_list) in embed_augment_face_dict.items():
        for embed in embed_list:
            train_X.append(embed)
        train_y = np.append(train_y, np.full(len(embed_list), label))
    train_X = np.asarray(train_X)
    
    # Train a linear SVM for facial recognition
    model = SVC(kernel='linear', probability=True)
    model.fit(train_X, train_y)
    
    return model

In [26]:
'''Run predictions using trained SVM on testing embedding set'''

def test_SVM(test_img_class_list, character_dict, model):
    for item in test_img_class_list:
        # Predict on the testing image set
        test_X = item.embedded_face_list
        pred_y = model.predict(test_X)
        item.add_predictions(pred_y)

In [27]:
''' Create clean prediction directories for all the characters '''

def make_prediction_directories(character_dict, path):
    for _, character in character_dict.items():
        
        prediction_path = path + '/' + character + '/prediction'
        # Create a fresh prediction directory for each character
        try:
            _ = os.makedirs(prediction_path)
        except FileExistsError:
            _ = shutil.rmtree(prediction_path)
            _ = os.makedirs(prediction_path)

In [28]:
''' Save all the predicted faces to the predicted character's project directory'''

def save_faces(test_img_class_list, path, character_dict):
    for item in test_img_class_list:
        for (face, label) in zip(item.face_list, item.prediction_list):
            
            # Find the name of the character associated with the label
            character = character_dict[label]
            
            # Save the face to the first available image number name
            i = 1
            prediction_path = path + '/' + character + '/prediction'
            while os.path.exists("{}/{}.png".format(prediction_path, i)):
                i += 1
            img_path = '{}/{}.png'.format(prediction_path, i)
            _ = cv2.imwrite(img_path, face)